In [ ]:
import praw
import json
import time
import os

# Initialize Reddit instance with credentials
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ", # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# List of subreddits focused on politics and marketing
politics_marketing_subreddits = [
    "politics", "PoliticalDiscussion", "conservative", "progressive", "neoliberal",
    "marketing", "advertising", "DigitalMarketing", "SEO", "content_marketing",
    "business", "smallbusiness", "Entrepreneur", "SocialMedia", "startups"
]

# Categories to retrieve posts from
categories = ["hot", "new", "rising", "controversial", "top"]

# Settings
posts_per_subreddit = 500
output_file = "redditdata2.json"
save_interval = 100  # Save data every 100 posts

# Load existing data and IDs from previous files to avoid duplicates
existing_ids = set()
data = []
for filename in ["reddit_data.json", "redditdata1.json", output_file]:
    if os.path.exists(filename):
        with open(filename, "r") as f:
            file_data = json.load(f)
            data.extend(file_data)
            existing_ids.update(post['id'] for post in file_data if 'id' in post)

# Fetch function with deduplication
def fetch_subreddit_posts(subreddit_name, category):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    try:
        submissions = getattr(subreddit, category)(limit=posts_per_subreddit)
        for submission in submissions:
            if submission.id not in existing_ids:  # Avoid duplicates
                post_data = {
                    "id": submission.id,
                    "title": submission.title,
                    "score": submission.score,
                    "author": str(submission.author),
                    "created_utc": submission.created_utc,
                    "subreddit": subreddit.display_name,
                    "selftext": submission.selftext,
                    "url": submission.url,
                    "category": category
                }
                posts.append(post_data)
                existing_ids.add(submission.id)
    except Exception as e:
        print(f"Error fetching {category} posts from r/{subreddit_name}: {e}")
    return posts

# Data collection with periodic saving
post_count = 0
try:
    for _ in range(50):  # Increased loop count to accumulate more data
        for subreddit_name in politics_marketing_subreddits:
            for category in categories:
                print(f"Fetching {category} posts from r/{subreddit_name}...")
                new_posts = fetch_subreddit_posts(subreddit_name, category)
                data.extend(new_posts)
                post_count += len(new_posts)
                time.sleep(2)  # Delay to respect Reddit's API rate limits

                # Save periodically to avoid data loss
                if post_count >= save_interval:
                    with open(output_file, "w") as f:
                        json.dump(data, f, indent=4)
                    print(f"Saved {post_count} posts to {output_file}")
                    post_count = 0  # Reset counter after save

except KeyboardInterrupt:
    print("Data collection interrupted. Saving progress...")

# Final save before exit
if post_count > 0:
    with open(output_file, "w") as f:
        json.dump(data, f, indent=4)
    print("Final save completed before exit.")

print(f"Data collection complete. Saved to {output_file}")


Fetching hot posts from r/politics...
Fetching new posts from r/politics...
Fetching rising posts from r/politics...
Fetching controversial posts from r/politics...
Fetching top posts from r/politics...
Fetching hot posts from r/PoliticalDiscussion...
Saved 510 posts to redditdata2.json
Fetching new posts from r/PoliticalDiscussion...
Fetching rising posts from r/PoliticalDiscussion...
Fetching controversial posts from r/PoliticalDiscussion...
Saved 481 posts to redditdata2.json
Fetching top posts from r/PoliticalDiscussion...
Saved 489 posts to redditdata2.json
Fetching hot posts from r/conservative...
Saved 500 posts to redditdata2.json
Fetching new posts from r/conservative...
Fetching rising posts from r/conservative...
Fetching controversial posts from r/conservative...
Saved 561 posts to redditdata2.json
Fetching top posts from r/conservative...
Saved 499 posts to redditdata2.json
Fetching hot posts from r/progressive...
Saved 500 posts to redditdata2.json
Fetching new posts from